# Analysis of agency behavioural outputs 

by Eliane Mueller July 2024. 
Analysis of the behavioural part of the Game of the neurofeedback project. First, the subjective judgement of performance (JoP) and the judgement of agency (JoA) are analysed, and then the objective performance (task performance, # tragets hit - # distractors hit)

The task contains the three conditions:
- Baseline: Cursor precisely follows the movement imposed by the input device. 
- Turbulence: Random noise was added to the cursor in 40% of the button presses. 
- Pure visual (ignored for the behavioural analysis)


In [ ]:
# Import packages
import pandas as pd, os, numpy as np, matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import ttest_ind, ttest_rel, wilcoxon, mannwhitneyu
from statistics import mean
from itertools import combinations
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as sfa
import scikit_posthocs as sp
import pingouin as pg

-----

## Set main paths & subject P codes

In [ ]:
# Save figures/results
save_to = './group_analysis/behav_results/Agency_game/'

# Path to txt outputs
# Data organised as: data/P-Code/Agency_task/Session/behavioural/...
main_path = './data/'

# Get the subject names
subj_list = [f for f in sorted(os.listdir(main_path)) if f.startswith('P')]

n_subj = len(subj_list)
print(len(subj_list))
print(subj_list)


## Extract main data Judgement of Performance / Judgement of Agency

In [ ]:
# Get txt files where JoP and JoA values are saved

# Get all txt files from session 1
filenames_V01 = list()
for i in range (len(subj_list)):
    filename = [f for f in sorted(os.listdir(main_path + subj_list[i] + '/V01'+'/2_Agency_Game/'+ 'behav/')) if (f.endswith('txt')) & ('Resu'in f) & (f.startswith('P'))]
    filenames_V01.extend(filename)

# Get all files from session 3
filenames_V03 = list()
for i in range (len(subj_list)):
    filename = [f for f in sorted(os.listdir(main_path + subj_list[i]+ '/V03'+'/4_Agency_Game/'+ 'behav/')) if (f.endswith('txt')) & ('Resu'in f) & (f.startswith('P'))]
    filenames_V03.extend(filename)
    
filenames = filenames_V01 + filenames_V03

print(filenames)

In [7]:
# Calculate JoP and JoA scores (covert scores to 11-Point Likert scale)

JoP_list = list()
JoA_list = list()
JoP_list_all_subj = list()
JoA_list_all_subj = list()
game_phase = ('Baseline','Turbulence')



for l in range(len(filenames)):
    # Load the file
    if l < n_subj:
        f = l
        df = pd.read_csv(main_path + subj_list[f] + '/V01' +'/2_Agency_Game/' + 'behav/' + filenames[f], sep='\t')
    else:
        f = l - n_subj
        df = pd.read_csv(main_path + subj_list[f] + '/V03' +'/4_Agency_Game/' + 'behav/' + filenames[l], sep='\t')
        
    df.columns = df.columns.str.replace(' ', '')
    df = df.astype(float)

    # Get pcode
    pcode = filenames[f].split('_')[0]
    
    # Exclude Pure Visual
    data = df[df.GamePhases !=1.0]
    
    # build mask per condition
    # PV: 1, BE:2, BH:3 , TE:4, TH:5
    # in this case BE & BH as well as TE & TH are exactly the same.
    base_mask = (data.GamePhases == 2) | (data.GamePhases == 3) 
    turb_mask = (data.GamePhases == 4) | (data.GamePhases == 5) 
    masks = [base_mask, turb_mask]
     
    
    JoP_list = list()
    JoA_list = list()
    
# Convert scores to 0-11 scale

    for i in range(len(masks)):
        JoP = data[masks[i]].JoP.values.tolist()
        JoA = data[masks[i]].JoA.values.tolist()
    
        for ele in range(len(JoP)):
            
            if  JoP[ele] == 390:
                JoP[ele] = 5
            elif JoP[ele] == 312:
                JoP[ele] = 4
            elif JoP[ele] == 234:
                JoP[ele] = 3
            elif JoP[ele] == 156:
                JoP[ele] = 2
            elif JoP[ele] == 78:
                JoP[ele] = 1
            elif JoP[ele] == 0:
                JoP[ele] = 0
            elif JoP[ele] == -78:
                JoP[ele] = -1
            elif JoP[ele] == -156:
                JoP[ele] = -2
            elif JoP[ele] == -234:
                JoP[ele] = -3
            elif JoP[ele] == -312:
                JoP[ele] = -4
            elif JoP[ele] == -390:
                JoP[ele] = -5

        for ele in range(len(JoA)):
            
            if  JoA[ele] == 390:
                JoA[ele] = 5
            elif JoA[ele] == 312:
                JoA[ele] = 4
            elif JoA[ele] == 234:
                JoA[ele] = 3
            elif JoA[ele] == 156:
                JoA[ele] = 2
            elif JoA[ele] == 78:
                JoA[ele] = 1
            elif JoA[ele] == 0:
                JoA[ele] = 0
            elif JoA[ele] == -78:
                JoA[ele] = -1
            elif JoA[ele] == -156:
                JoA[ele] = -2
            elif JoA[ele] == -234:
                JoA[ele] = -3
            elif JoA[ele] == -312:
                JoA[ele] = -4
            elif JoA[ele] == -390:
                JoA[ele] = -5
            
        
        JoP_list.append (JoP) #Append both baseline and turbulence JoP to JoP_list
        JoA_list.append (JoA) #Append both baseline and turbulence JoA to JoA_list
        
        
    JoP_list_all_subj.append([pcode,JoP_list, list(game_phase)])
    JoA_list_all_subj.append([pcode,JoA_list, list(game_phase)])
    


## Judgement of Agency plots and stat testing (group-level)

In [ ]:
# Create a box plot for the whole group of agency ratings for both sessions (pre and post NF) and each condition (turbulence & baseline).

plt.style.use('default')
save_plots_to = './group_analysis/behav_results/Agency_game/judgement_indiv/'

v1_l0=list()
v1_l1=list()
v2_l0=list()
v2_l1 = list()

for s in range(n_subj):
    # Get all the scores
    s_data1 = JoA_list_all_subj[s] # Session 1 data
    s_data2 = JoA_list_all_subj[s+n_subj]# Session 2 data
    s_values1 = s_data1[1:][0]      # these are the scores
    s_values2 = s_data2[1:][0]
    diff_level_names = s_data1[-1] # these are the diff levels that correspond to the scores
    
    v1_l0.append(np.median(s_values1[0])) # lvl 0 
    v1_l1.append(np.median(s_values1[1])) # lvl 1
    v2_l0.append(np.median(s_values2[0])) # lvl 0
    v2_l1.append(np.median(s_values2[1])) # lvl 1
    
  
v1_l0_all = np.array(v1_l0) 
v1_l1_all = np.array(v1_l1) 
v2_l0_all = np.array(v2_l0)
v2_l1_all = np.array(v2_l1)

    
s_all1 = [v1_l0_all, v1_l1_all]
print(s_all1)
s_all2 = [v2_l0_all, v2_l1_all]

# t-test on differences 
v1_l0_all = pd.DataFrame(v1_l0_all)# create data frames
v1_l1_all = pd.DataFrame(v1_l1_all)
v2_l0_all = pd.DataFrame(v2_l0_all)
v2_l1_all = pd.DataFrame(v2_l1_all)

# Paired t-test between the two conditions of the first session (pre-NF)
result = ttest_rel(v1_l0_all, v1_l1_all)
# print result
print('Paired t-Test Pre-NF:', result)

# Paired t-test between the two conditions of the first session (post-NF)
result = ttest_rel(v2_l0_all, v2_l1_all)
print('Paired t-Test Post-NF:', result)
    
x_labels = ['Pre-NF', 'Post-NF']
    
# Figure
plt.figure(dpi=120)
    
# First boxplot pair 
bp1 = plt.boxplot (s_all1, positions = [1,2], widths = 0.6,patch_artist = True, showmeans = True, meanprops=dict(marker='o', markerfacecolor='black', markersize=4, markeredgecolor='black'))
bp2 = plt.boxplot (s_all2, positions = [4,5], widths = 0.6,patch_artist = True, showmeans = True, meanprops=dict(marker='o', markerfacecolor='black', markersize=4, markeredgecolor='black'))
   
# bplot = plt.boxplot(s_values, patch_artist=True, showmeans=True)
plt.ylabel('Judgement of Agency', size=17)
plt.yticks(range (-5,6))
plt.xlabel('Session', size=17)
plt.xticks((1.5,4.5), x_labels, size = 14)
sns.despine(top=True, right=True, left=False, bottom=False)

plt.title('Judgement of Agency (n=18)', size = 20, pad = 50)

# fill with colors
colors = ['cadetblue', 'lightblue', 'cadetblue', 'lightblue']
for bp in [bp1, bp2]:
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        #patch.set_alpha(0.6)
        #patch.set(hatch = '')
    for patch, color in zip(bp['medians'], colors):
        patch.set(color = 'k', linewidth=2)
        patch.set_alpha(0.6)

hcadetblue, = plt.plot(1, 1, 's', color='cadetblue', markersize=12, label='Baseline')
hLightBlue, = plt.plot(1, 1, 's', color='lightblue', markersize=12, label='Turbulence')
legend((hcadetblue, hLightBlue),('Baseline', 'Turbulence'))
legend(bbox_to_anchor=(1.3, 0.55))

hcadetblue.set_visible(False)
hLightBlue.set_visible(False)

plt.savefig(save_plots_to + 'JoA_all', dpi=200)

In [ ]:
# Stat testing of within session comparison and change in differences - Wilcoxon signed rank test
print(f" -> We have {n_subj} subjects in the working directory.")

v1_l0 = []
v1_l1 = []
v2_l0 = []
v2_l1 = []
v1_diff = []
v2_diff = []

for s in range(n_subj):
    s_data1 = JoA_list_all_subj[s]  # Session 1 data
    gamees = (s_data1[1:][0])
    base = gamees[0]
    turb = gamees[1]

    mean_base = np.median(base)
    mean_turb = np.median(turb)

    v1_l0.append(mean_base)
    v1_l1.append(mean_turb)
    
    diff = mean_base - mean_turb
    v1_diff.append(diff)
    
    s_data2 = JoA_list_all_subj[s+n_subj]  # Session 2 data
    gamees2 = (s_data2[1:][0])
    base2 = gamees2[0]
    turb2 = gamees2[1]

    mean_base2 = np.median(base2)
    mean_turb2 = np.median(turb2)

    v2_l0.append(mean_base2)
    v2_l1.append(mean_turb2)
    
    diff2 = mean_base2 - mean_turb2
    v2_diff.append(diff2)


# Test condition difference within each session
stat_pre, p_pre = wilcoxon(v1_l0,v1_l1)
stat_post, p_post = wilcoxon(v2_l0,v2_l1)
print(f"Pre-session Turbulence vs. Baseline: p = {p_pre:.4f}")
print(f"Post-session Turbulence vs. Baseline: p = {p_post:.4f}")

# Test difference between the differences (i.e., interaction)
stat_interact, p_interact = wilcoxon(v1_diff,v2_diff)
print(f"Difference of differences (interaction): p = {p_interact:.4f}")

pvals = [p_pre, p_post, p_interact]
rej, pvals_corr, _, _ = multipletests(pvals, method='holm')
print(f"Corrected p-values: {pvals_corr}")

### Judgement of Agency - Violin Plot

In [ ]:

# Flatten and prepare the data for plotting
v1_l0_all_flat = v1_l0_all.squeeze()
v1_l1_all_flat = v1_l1_all.squeeze()
v2_l0_all_flat = v2_l0_all.squeeze()
v2_l1_all_flat = v2_l1_all.squeeze()

# Create a DataFrame for seaborn with each condition/session combination as its own category
df_vp = pd.DataFrame({
    'Judgement of Agency': pd.concat([v1_l0_all_flat, v1_l1_all_flat, v2_l0_all_flat, v2_l1_all_flat], ignore_index=True),
  'Session': ['Pre-NF'] * len(v1_l0_all_flat) + ['Pre-NF'] * len(v1_l1_all_flat) + ['Post-NF'] * len(v2_l0_all_flat) + ['Post-NF'] * len(v2_l1_all_flat),
    'Condition': ['Non-Turbulence'] * len(v1_l0_all_flat) + ['Turbulence'] * len(v1_l1_all_flat) + ['Non-Turbulence'] * len(v2_l0_all_flat) + ['Turbulence'] * len(v2_l1_all_flat)
})

# Set the plot style
plt.style.use('default')
plt.figure(dpi=300)


# Plot violin plot for each condition/session combination as separate violins
ax = sns.violinplot(
    x='Session', 
    y='Judgement of Agency', 
    data=df_vp, 
    hue='Condition',
    palette={'Non-Turbulence': 'white', 'Turbulence': 'lightgrey', 'Non-Turbulence': 'white', 'Turbulence': 'lightgrey'}, 
    cut=0.4, inner="box", linecolor = "black"  # turn off inner boxplot to plot manually with more control
)

# Customize plot labels and title
plt.ylabel('JoA', size=17)
plt.yticks(range(-5, 6))
plt.xlabel('Session', size=17)
plt.xticks(size=14)
sns.despine(top=True, right=True, left=False, bottom=False)
plt.title('Judgement of Agency (n=18)', size=20, pad=50)

# Custom Legend - Get handles and labels from the plot
handles, labels = ax.get_legend_handles_labels()

# Only select the first two elements (Baseline and Turbulence)
new_handles = handles[:2]
new_labels = labels[:2]

# Create the custom legend
plt.legend(new_handles, new_labels, bbox_to_anchor=(1.05, 1), loc='upper left')

# Save figure
# plt.savefig(save_plots_to + 'JoA_violin_separated', dpi=200)
# plt.show()

## Judgement of Performance plots and stat testing (group-level)

In [ ]:
# Flatten and prepare the data for plotting
v1_l0_p_all_flat = v1_l0_p_all.squeeze()
v1_l1_p_all_flat = v1_l1_p_all.squeeze()
v2_l0_p_all_flat = v2_l0_p_all.squeeze()
v2_l1_p_all_flat = v2_l1_p_all.squeeze()

# Create a DataFrame for seaborn with each condition/session combination as its own category
df_vp = pd.DataFrame({
    'Judgement of Perf': pd.concat([v1_l0_p_all_flat, v1_l1_p_all_flat, v2_l0_p_all_flat, v2_l1_p_all_flat], ignore_index=True),
  'Session': ['Pre-NF'] * len(v1_l0_p_all_flat) + ['Pre-NF'] * len(v1_l1_p_all_flat) + ['Post-NF'] * len(v2_l0_p_all_flat) + ['Post-NF'] * len(v2_l1_p_all_flat),
    'Condition': ['Non-Turbulence'] * len(v1_l0_p_all_flat) + ['Turbulence'] * len(v1_l1_p_all_flat) + ['Non-Turbulence'] * len(v2_l0_p_all_flat) + ['Turbulence'] * len(v2_l1_p_all_flat)
})

# Set the plot style
plt.style.use('default')
plt.figure(dpi=300)


# Plot violin plot for each condition/session combination as separate violins
ax = sns.violinplot(
    x='Session', 
    y='Judgement of Perf', 
    data=df_vp, 
    hue='Condition',
    palette={'Non-Turbulence': 'white', 'Turbulence': 'lightgrey', 'Non-Turbulence': 'white', 'Turbulence': 'lightgrey'}, 
    cut=0.4, inner="box", linecolor = "black"  # turn off inner boxplot to plot manually with more control
)

# Customize plot labels and title
plt.ylabel('JoP', size=17)
plt.yticks(range(-5, 6))
plt.xlabel('Session', size=17)
plt.xticks(size=14)
sns.despine(top=True, right=True, left=False, bottom=False)
plt.title('Judgement of Performance (n=18)', size=20, pad=50)

# Custom Legend - Get handles and labels from the plot
handles, labels = ax.get_legend_handles_labels()

# Only select the first two elements (Baseline and Turbulence)
new_handles = handles[:2]
new_labels = labels[:2]

# Create the custom legend
plt.legend(new_handles, new_labels, bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
# Stat testing of within session comparison and change in differences - Wilcoxon signed rank test
print(f" -> We have {n_subj} subjects in the working directory.")

v1_p_l0 = []
v1_p_l1 = []
v2_p_l0 = []
v2_p_l1 = []
v1_p_diff = []
v2_p_diff = []

for s in range(n_subj):
    s_data1 = JoP_list_all_subj[s]  # Session 1 data
    gamees = (s_data1[1:][0])
    base = gamees[0]
    turb = gamees[1]

    mean_base = np.median(base)
    mean_turb = np.median(turb)

    v1_p_l0.append(mean_base)
    v1_p_l1.append(mean_turb)
    
    diff = mean_base - mean_turb
    v1_p_diff.append(diff)
    
    s_data2 = JoP_list_all_subj[s+n_subj]  # Session 2 data
    gamees2 = (s_data2[1:][0])
    base2 = gamees2[0]
    turb2 = gamees2[1]

    mean_base2 = np.median(base2)
    mean_turb2 = np.median(turb2)

    v2_p_l0.append(mean_base2)
    v2_p_l1.append(mean_turb2)
    
    diff2 = mean_base2 - mean_turb2
    v2_p_diff.append(diff2)


# Test condition difference within each session
stat_pre_perf, p_pre_perf = wilcoxon(v1_p_l0,v1_p_l1)
stat_post_perf, p_post_perf = wilcoxon(v2_p_l0,v2_p_l1)
print(f"Pre-session Turbulence vs. Baseline: p = {p_pre_perf:.4f}")
print(f"Post-session Turbulence vs. Baseline: p = {p_post_perf:.4f}")

# Test difference between the differences (i.e., interaction)
stat_interact_perf, p_interact_perf = wilcoxon(v1_p_diff,v2_p_diff)
print(f"Difference of differences (interaction): p = {p_interact_perf:.4f}")

pvals_perf = [p_pre_perf, p_post_perf, p_interact_perf]
rej, pvals_corr_perf, _, _ = multipletests(pvals_perf, method='holm')
print(f"Corrected p-values: {pvals_corr_perf}")

# Task performance - group level plots and stat testing

In [ ]:
# Net performance / real performance plot and stat testing
plt.style.use('default')
save_to = './data/behav_results/Agency_game'

# first extact all the # of traghets hit and # of distractors hit for each subject and each session and save as a list (v1_l0_all_hitx, v1_l0_all_hitO, etc)
    
v1_l0_all_np = v1_l0_all_hitx - v1_l0_all_hitO
v1_l1_all_np = v1_l1_all_hitx - v1_l1_all_hitO
v2_l0_all_np = v2_l0_all_hitx - v2_l0_all_hitO
v2_l1_all_np = v2_l1_all_hitx - v2_l1_all_hitO
 
s_all1_np = [v1_l0_all_np, v1_l1_all_np]
s_all2_np = [v2_l0_all_np, v2_l1_all_np]

x_labels = ['Pre-NF', 'Post-NF']
    
# Figure
plt.figure(dpi=120)
    
# First boxplot pair 
bp1 = plt.boxplot (s_all1_np, positions = [1,2], widths = 0.6,patch_artist = True, showmeans = True, meanprops=dict(marker='o', markerfacecolor='black', markersize=4, markeredgecolor='black'))
    
bp2 = plt.boxplot (s_all2_np, positions = [4,5], widths = 0.6,patch_artist = True, showmeans = True, meanprops=dict(marker='o', markerfacecolor='black', markersize=4, markeredgecolor='black'))

plt.ylabel('Real performance [%HitX - %HitO]', size=14)
plt.yticks(range (0,100,10))
plt.xlabel('Session', size=17)
plt.xticks((1.5,4.5), x_labels, size = 14)
sns.despine(top=True, right=True, left=False, bottom=False)

plt.title('Real performance (n=18)', size = 20, pad = 50)


# fill with colors
colors = ['mistyrose', 'salmon', 'mistyrose', 'salmon']
for bp in [bp1, bp2]:
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        #patch.set_alpha(0.6)
        #patch.set(hatch = '')
    for patch, color in zip(bp['medians'], colors):
        patch.set(color = 'k', linewidth=2)
        patch.set_alpha(0.6)

hBeige, = plt.plot(1, 1, 's', color='mistyrose', markersize=12, label='Baseline')
hLightBlue, = plt.plot(1, 1, 's', color='salmon', markersize=12, label='Turbulence')
legend((hBeige, hLightBlue),('Baseline', 'Turbulence'))
legend(bbox_to_anchor=(1.3, 0.55))
hBeige.set_visible(False)
hLightBlue.set_visible(False)

#plt.savefig(save_plots_to + 'JoP_all', dpi=200)

In [ ]:
# Real performance stat testing (ANOVA)

# Subjects IDs
subjects = np.arange(18)

v1_l0_all_np = pd.Series(v1_l0_all_np)
v1_l1_all_np = pd.Series(v1_l1_all_np)
v2_l0_all_np = pd.Series(v2_l0_all_np)
v2_l1_all_np = pd.Series(v2_l1_all_np)

df_long_np = pd.DataFrame({
    'Subject': np.tile(subjects,4),
    'Session': ['Pre']*18 + ['Pre']*18 + ['Post']*18 + ['Post']*18,
    'Condition': ['Baseline']*18 + ['Turbulence']*18 + ['Baseline']*18 + ['Turbulence']*18,
    'Value': pd.concat([v1_l0_all_np,v1_l1_all_np,v2_l0_all_np,v2_l1_all_np])
})

anova_np = pg.rm_anova(dv = 'Value', within = ['Session', 'Condition'], subject = 'Subject', data = df_long_hitx, detailed = True)
print(anova_np)

posthoc_np = pg.pairwise_tests(dv='Value', within=['Session', 'Condition'],
                             subject='Subject', data=df_long_np,
                             padjust='bonf', effsize='cohen', return_desc = True)
print(posthoc_np)


In [ ]:
# Flatten and prepare data for seaborn
data = []
groups = []
sessions = []
conditions = []

# Helper to structure the data
def add_data(values, group, session, condition):
    for v in values:
        data.append(v)
        groups.append(group)
        sessions.append(session)
        conditions.append(condition)

# First partition the data into responders and non-responders and create separate lists for each group and condition

# Add responder data
add_data(v1_l0_res_all, 'Responder', 'Pre', 'Non-Turbulence')
add_data(v1_l1_res_all, 'Responder', 'Pre', 'Turbulence')
add_data(v2_l0_res_all, 'Responder', 'Post', 'Non-Turbulence')
add_data(v2_l1_res_all, 'Responder', 'Post', 'Turbulence')

# Add non-responder data
add_data(v1_l0_nonres_all, 'Non-responder', 'Pre', 'Non-Turbulence')
add_data(v1_l1_nonres_all, 'Non-responder', 'Pre', 'Turbulence')
add_data(v2_l0_nonres_all, 'Non-responder', 'Post', 'Non-Turbulence')
add_data(v2_l1_nonres_all, 'Non-responder', 'Post', 'Turbulence')

# Create DataFrame
df = pd.DataFrame({
    'Performance': data,
    'Group': groups,
    'Session': sessions,
    'Condition': conditions
})

# Create combined column for hue
df['Session_Condition'] = df['Session'] + ' - ' + df['Condition']

# Define color palette
palette = {
    'Turbulence': 'lightgrey',
    'Non-Turbulence': 'white',

}

# Set up subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

# Responder plot
sns.violinplot(
    x='Session_Condition', y='Performance',
    data=df[df['Group'] == 'Responder'],
    palette=palette, hue = 'Condition', ax=axes[0],
    cut=0.5, inner='box', width = 0.8, linecolor = "black"
)
axes[0].set_title('Responders', fontsize=18, pad = 60)
axes[0].set_xlabel('')
axes[0].set_ylabel('Task Performance [%]', fontsize=14)
axes[0].tick_params(axis='x', labelsize=12)
axes[0].legend(loc = 'lower right', fontsize = 12)
axes[0].set_xticks([0.5,2.5])
axes[0].set_xticklabels (['Pre-NF','Post-NF'], size = 14)

# Non-responder plot
sns.violinplot(
    x='Session_Condition', y='Performance',
    data=df[df['Group'] == 'Non-responder'],
    palette=palette, hue = 'Condition', ax=axes[1],
    cut=0.5, inner='box', linecolor= "black"
)
axes[1].set_title('Non-Responders', fontsize=18, pad = 60)
axes[1].set_xlabel('')
axes[1].set_ylabel('')
axes[1].tick_params(axis='x',  labelsize=12)
axes[1].legend(loc = 'lower right', fontsize =12)
axes[1].set_xticks([0.5,2.5])
axes[1].set_xticklabels (['Pre-NF','Post-NF'], size = 14)


# General formatting
fig.suptitle('Performance Comparison by Group and Condition', fontsize=20)
sns.despine()
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()